In [ ]:
from bastionlab import Identity

# Create `Identity` for Data owner.
data_owner = Identity.create("data_owner")

In [ ]:
!wget 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'

In [ ]:
!python datagen.py

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!unzip smsspamcollection.zip

In [ ]:
!pip install transformers==0.13.2

In [1]:
from tokenizers import Tokenizer
from bastionlab.polars.policy import Policy, Aggregation, Log
import polars as pl
from bastionlab import Connection

file_path = "./SMSSpamCollection"

labels = []
texts = []
with open(file_path) as f:
    for line in f.readlines():
        split = line.split("\t")
        labels.append(1 if split[0] == "spam" else 0)
        texts.append(split[1])
df = pl.DataFrame({"label": labels, "text": texts})

connection = Connection("localhost")


policy = Policy(safe_zone=Aggregation(min_agg_size=10), unsafe_handling=Log())
rdf = connection.client.polars.send_df(
    df.limit(int(df.height * 0.8)).limit(64), policy=policy, sanitized_columns=["text"]
)
test_rdf = connection.client.polars.send_df(
    df.tail(int(df.height * 0.2)).limit(64), policy=policy, sanitized_columns=["text"]
)


tokenizer = Tokenizer.from_pretrained("distilbert-base-uncased")
tokenizer.enable_truncation(max_length=32)
tokenizer.enable_padding(length=32)

train_rds = (
    rdf.convert(["text"], tokenizer.to_str())
    .collect()
    .to_dataset(["text_ids", "text_mask"], "label")
)
test_rds = (
    test_rdf.convert(["text"], tokenizer.to_str())
    .collect()
    .to_dataset(["text_ids", "text_mask"], "label")
)


train_rds._set_test_dataset(test_rds.train_dataset_ref)
test_rds.set_train_dataset(train_dataset=train_rds.train_dataset_ref)

In [ ]:
! pip install transformers

In [2]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from bastionlab.torch.utils import MultipleOutputWrapper

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
    torchscript=True,
)
model = MultipleOutputWrapper(model, 0)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

In [3]:
remote_datasets = connection.client.torch.list_remote_datasets()
print(remote_datasets)

remote_datasets[0].trace_input

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0])]

In [4]:
from bastionlab.torch.optimizer_config import Adam
remote_learner = connection.client.torch.RemoteLearner(
    model,
    remote_datasets[0],
    max_batch_size=2,
    loss="cross_entropy",
    optimizer=Adam(lr=5e-5),
    model_name="DistilBERT",
    expand=False,
)

Sending DistilBERT: 100%|████████████████████| 268M/268M [00:11<00:00, 24.1MB/s] 


In [5]:
remote_learner.fit(nb_epochs=2, eps=None)

GRPCException: Internal server error: code=StatusCode.INTERNAL message=Torch error: Internal torch error: unsupported operation: more than one element of the written-to tensor refers to a single memory location. Please clone() the tensor before performing the operation.
Exception raised from assert_no_internal_overlap at ../aten/src/ATen/MemoryOverlap.cpp:38 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >) + 0x6b (0x7f245903d9cb in /home/kbamponsem/libtorch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, char const*) + 0xd1 (0x7f2459039501 in /home/kbamponsem/libtorch/lib/libc10.so)
frame #2: <unknown function> + 0x11f8537 (0x7f245a281537 in /home/kbamponsem/libtorch/lib/libtorch_cpu.so)
frame #3: at::TensorIteratorBase::compute_mem_overlaps(at::TensorIteratorConfig const&) + 0x83 (0x7f245a2b9e33 in /home/kbamponsem/libtorch/lib/libtorc